#News Headlines of India Classifier

Dataset Attribution:
    https://www.kaggle.com/therohk/india-headlines-news-dataset

In [ ]:
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
categories = {}
with open("./india-news-headlines.csv", "r", encoding="utf8") as f:
    text = f.read()
    lines = text.split("\n")
    print(len(lines[1:-1]))
    for line in lines[1:-1]:
        parts = line.split(",")
        if (parts[1] not in categories):
            categories[parts[1]] = []
        categories[parts[1]].append(", ".join(parts[2:]))

Aim below (3 and 4) is to roll up sub-categories into broader categories. May be better to do this by hand eventually... should provide a better output. 

In [ ]:
arr = []
final_cats = {}
for key in categories:
    arr.append((key, len(key.split("."))))
    
arr = sorted(arr, key = lambda x:x[1])

for i in range(len(arr)):
    if (arr[i][1] < 3):
        final_cats[arr[i][0]] = []

for i in range(len(arr)):
    if (arr[i][1] >= 3):
        parts = arr[i][0].split(".")
        base_phrase = ".".join(parts[0:2])
        num_found = 0
        for k in range(i+1, len(arr)):
            if (arr[k][1] >= 3 and k != i):
                if (base_phrase in arr[k][0]):
                    num_found += 1
        if (num_found > 0):
            if (base_phrase not in final_cats):
                final_cats[base_phrase] = []
            final_cats[base_phrase].append(arr[i][0])

In [ ]:
exclude = ["city", "times", "unknown", "removed", "cities", "top-headlines", "top-stories"]
count = 0
redo = True
while (redo):
    redo = False
    del_list = {}
    for key in final_cats:
        include = True
        for i in range(len(exclude)):
            if (exclude[i] in key):
                include = False
        if (include):
            total = 0
            if (key in categories):
                total += len(categories[key])
            if (len(final_cats[key]) > 0):
                for i in range(len(final_cats[key])):
                    if (final_cats[key][i] in categories):
                        total += len(categories[final_cats[key][i]])
            if (total > 5000):
                count += 1
            else:
                parts = key.split(".")
                use_key = parts[0]
                if ("cricket" in key or "ipl" in key or "icc" in key): #this is hacky but it's the only way to keep the rest of the code clean
                    use_key = "cricket"
                if (parts[0] in final_cats and len(parts) > 1):
                    del_list[key] = use_key
                    redo = True
    
    print("Merging:", len(del_list.keys()))
    for key in del_list:
        if (del_list[key] not in final_cats):
            final_cats[del_list[key]] = []
        final_cats[del_list[key]].extend(final_cats[key])
        if (key in categories):
            final_cats[del_list[key]].append(key)
        del final_cats[key]

good_cats = []
for key in final_cats:
    include = True
    for i in range(len(exclude)):
        if (exclude[i] in key):
            include = False
    if (include):
        total = 0
        if (key in categories):
            total += len(categories[key])
        if (len(final_cats[key]) > 0):
            for i in range(len(final_cats[key])):
                if (final_cats[key][i] in categories):
                    total += len(categories[final_cats[key][i]])
        if (total > 2500): #select categories with more than 2500 datapoints
            good_cats.append(key)

In [ ]:
#again, hacky but required
final_cats["cricket"].append("sports.icc-world-cup-2015")
final_cats["cricket"].extend(final_cats["sports.icc-world-cup-2015"])
del(good_cats[good_cats.index("sports.icc-world-cup-2015")])

In [ ]:
print(good_cats, len(good_cats))

In [ ]:
reorg = {}

for i in range(len(good_cats)):
    key = good_cats[i]
    if (key not in reorg):
        reorg[key] = []
    reorg[key].extend(categories[key])
    for sub_key in final_cats[key]:
        reorg[key].extend(categories[sub_key])
        
for key in reorg:
    print(key, len(reorg[key]))

In [ ]:
original_distribution = categories
categories = reorg
good_categories = list(categories.keys())
num_cat = len(good_categories)

In [ ]:
import re
def processHeadline_One(text):
    return re.sub(r'[^\w\s]','',text)

In [ ]:
longest_phrase = 0
word_lengths = []
vocab = {}
for i in range(len(good_categories)):
    t_cat = good_categories[i]
    for k in range(len(categories[t_cat])):
        headline = processHeadline_One(categories[t_cat][k])
        words = headline.split(" ")        
        for j in range(len(words)):
            if (words[j] not in vocab):
                vocab[words[j]] = 0
            vocab[words[j]] += 1
        word_lengths.append(len(words))
print(np.mean(word_lengths), np.percentile(word_lengths, 99))

In [ ]:
vocab_arr = sorted([(word, vocab[word]) for word in vocab], key=lambda x:x[1], reverse=True)

In [ ]:
final_vocab = {}
rev_vocab = {}
for i in range(len(vocab_arr)):
    final_vocab[vocab_arr[i][0]] = i
    rev_vocab[i] = vocab_arr[i][0]

In [ ]:
print(len(vocab_arr))
print(vocab_arr[75000:75020])

In [ ]:
#build dataset
vocab_cutoff = 75000
alt_words = {}
alt_words["#UNKNOWN_WORD#"] = vocab_cutoff + 1
alt_words["#EOS"] = vocab_cutoff + 2
alt_words["#PADDING_WORD#"] = vocab_cutoff + 3

In [ ]:
def word_index(word): #returns the lowest possible index for a word, regardless of case
    index = alt_words["#UNKNOWN_WORD#"]
    if (word in final_vocab):
        index = final_vocab[word]
    try:
        if (final_vocab[word.lower()] < index):
            index = final_vocab[word.lower()]
        if (final_vocab[word.upper()] < index):
            index = final_vocab[word.upper()]
        n_word = word[0].upper() + word[1:]
        if (final_vocab[n_word] < index):
            index = final_vocab[n_word]
    except:
        if (word in final_vocab):
            index = final_vocab[word]
    if (index > vocab_cutoff):
        index = alt_words["#UNKNOWN_WORD#"]
    return index

In [ ]:
dataset = []

for i in range(len(good_categories)):
    headlines = categories[good_categories[i]]
    for k in range(len(headlines)):
        headline = processHeadline_One(headlines[k])
        indices = []
        words = headline.split(" ")
        if (len(words) <= 14):
            for x in range(len(words)):
                indices.append(word_index(words[x]))
            indices.append(alt_words["#EOS"])
            while (len(indices) < 15):
                indices.append(alt_words["#PADDING_WORD#"])
            dataset.append((indices,i))         

In [ ]:
print(len(dataset))
dataset = np.array(dataset)
np.random.shuffle(dataset)

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Dense, GRU, LSTM, Dropout

In [ ]:
input_layer = Input((15,))
embedding = Embedding(75005, 128)(input_layer)
gru_1 = LSTM(128)(embedding)
dropout = Dropout(0.1)(gru_1)
d1 = Dense(128, activation="tanh")(dropout)
d2 = Dense(64, activation="relu")(d1)
output = Dense(num_cat, activation="sigmoid")(d2) #sigmoid used here to allow for extraction of multiple topics

model = Model(input_layer, output)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics = ["categorical_accuracy"])
print(model.summary())

In [ ]:
validation_split = 0.1
val_cutoff = int(validation_split * len(dataset))
train = dataset[val_cutoff:]
validate = dataset[:val_cutoff]

x_train = [train[i][0] for i in range(len(train))]
y_train = [train[i][1] for i in range(len(train))]

x_val = [validate[i][0] for i in range(len(validate))]
y_val = [validate[i][1] for i in range(len(validate))]

print(len(x_train))

In [ ]:
val_acc = 0

In [ ]:
step = 50000
num_full_epochs = 4
for k in range(num_full_epochs): 
    print("Restarting on full dataset...", k)
    for i in range(0, 750000, step):
        print("epoch:", k, ",", i, ":", i+step)
        x = [] 
        y = []
        for j in range(i, i+step):
            x.append(np.asarray(x_train[j]))
            arr = np.zeros(num_cat, dtype=np.int8)
            arr[y_train[j]] = 1
            y.append(arr)
        model.fit(np.asarray(x), np.asarray(y), batch_size=128, validation_split=0.1)
    print("\n\n\n\n Predicting:")
    x_ = []
    y_ = []
    for i in range(0, len(x_val)):
        x_.append(np.asarray(x_val[i]))
    output = model.predict(np.asarray(x_), verbose=True)
    indices = np.argmax(output, axis=1)
    acc = 0
    for i in range(len(indices)):
        if (indices[i] == y_val[i]):
            acc += 1
    accuracy = acc/len(indices)
    if (accuracy > val_acc):
        print("\nValidation Accuracy went up to:", accuracy, "\nSaving model.\n\n\n\n")
        model.save("./HeadlinesOfIndia.h5")
        val_acc = accuracy
    else:
        print("\nNew validation accuracy is:", accuracy, "\nNot saving.\n\n\n\n")

In [ ]:
print(val_acc)

In [ ]:
for i in range(0, len(x_val)):
    x_.append(np.asarray(x_val[i]))
output = model.predict(np.asarray(x_), verbose=True)

In [ ]:
rev_vocab[75001] = "#UNKNOWN_WORD#"
rev_vocab[75002] = "#EOS"
for i in range(7000, 8000): #len(x_val)):
    sent = x_val[i]
    sent_str = ""
    for word in sent:
        if (word < 75002):
            sent_str += rev_vocab[word] + " "
    cats = output[i]
    scores = sorted([(i,cats[i]) for i in range(len(cats))], key=lambda x: x[1], reverse=True)
    potential_cats = [good_categories[scores[k][0]] for k in range(1)]
    potential_cats = "[ " + " ; ".join(potential_cats) + " ]"
    print(sent_str, potential_cats)

Predict the full "Unknown" category

In [ ]:
test_dataset = []
test_categories = ["unknown"]
for i in range(len(test_categories)):
    headlines = original_distribution[test_categories[i]]
    for k in range(len(headlines)):
        headline = processHeadline_One(headlines[k])
        indices = []
        words = headline.split(" ")
        if (len(words) <= 14):
            for x in range(len(words)):
                indices.append(word_index(words[x]))
            indices.append(alt_words["#EOS"])
            while (len(indices) < 15):
                indices.append(alt_words["#PADDING_WORD#"])
            test_dataset.append((indices,i)) 

In [ ]:
x_test = [test_dataset[i][0] for i in range(len(test_dataset))]

In [ ]:
x_ = []
for i in range(0, len(x_test)):
    x_.append(np.asarray(x_test[i]))
output = model.predict(np.asarray(x_), verbose=True)

In [ ]:
unknown_categorised = []

rev_vocab[75001] = "#UNKNOWN_WORD#"
rev_vocab[75002] = "#EOS"
for i in range(len(x_test)):
    sent = x_test[i]
    sent_str = ""
    for word in sent:
        if (word < 75002):
            sent_str += rev_vocab[word] + " "
    new_headline = {}
    new_headline["headline"] = sent_str

    cats = output[i]
    scores = sorted([(i,cats[i]) for i in range(len(cats))], key=lambda x: x[1], reverse=True)
    potential_cats = [good_categories[scores[k][0]] for k in range(3)]
    new_headline["categories"] = potential_cats
    unknown_categorised.append(new_headline)
    #potential_cats = "[ " + " ; ".join(potential_cats) + " ]"
    #print(sent_str, potential_cats)

In [ ]:
import json
json_str = json.dumps(unknown_categorised, indent=4)
with open("./testing_category_unknown.json", "w") as f:
    json.dump(unknown_categorised, f, indent=4)